In [2]:
#Zipline iExpense Download
#-------------------------
#libs
import hashlib
import requests
import time
import xlrd 
import warnings
warnings.filterwarnings("ignore")

print("Opening File...")

#Read Excel File
#loc = (" expense Jan to June 2018 cleaned.xlsx") 
loc = ("/Volumes/GoogleDrive/My Drive/Enterprise digital/Clients - in TEAM DRIVE/Google/Zipline/expense Jan to June 2018 cleaned.xlsx")
wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(0) 
sheet.cell_value(0, 0) 

print("File Opened.")
print("Downloading attachments...")

attNone =0
attSingle =0
attMult =0

#For each row
for i in range(68400,100000):
    #display status
    if i % 100 == 0:
        print(str(i) + "/100000 items parsed",end="\r"),
        
    #get fields
    expenseIdDetail = str(sheet.cell_value(i,0))[:-2]
    expenseId = str(sheet.cell_value(i,1))[:-2]
    unixTime = str(int(time.time()))
    
    #run encryption
    subStringURL = "@pintExpenseDtlID=" + expenseIdDetail + "&@pvcErrMsg=&" + unixTime
    hexOutput = hashlib.md5(subStringURL.encode('ascii')).hexdigest()

    #HTTP Request to get file ID
    url = "https://ipowerau.eps.ad.pwcinternal.com/GUI/iExpense/dset/xmlfEXPiExpenseFileReceiptsGet.dset"
    querystring = "@pintExpenseDtlID="+expenseIdDetail+"&@pvcErrMsg=&"+unixTime+"&"+hexOutput
    payload = ""
    headers = {
        'Accept': 'json',
        'JSON': 'true',
        'Cookie': "pageRows=10; ASP.NET_SessionId=nt5s1ebtlxuaobchtovnsxsr; PwCGTSiPOWER=30BFF0C17194FC33F989278DEC65F9C8B4D0775D72F4BB6024C7B243782A9DDA154E0E5CA1F110DC87EB150D011EF96ED2D27A350C82B638E8F1D58DA390CCD2EE8166B7AFD090DD6E7F951551F61F113A7A378CA6E52FA5D3D4627899DEE77E62B7030978050A8109BFEC8EA38D55543047014B435046E239652B0EB452CAA483C11A0B7DC0E4F54B8A533E718CA808727DAA11B76E965A20A5E429C90031E9683DBB44533FA6DD3A36FFA6F2E12668EF45235472AAB0A40B41D8C70728E4E0122CDD0EC747FD528D2776A2FFD69BA2D6832C32A59CFBA15A2026C22E4F066AB5F60D9BA894B884DB521065867D3033301FF2253A4041F2E9202BA75A1E2C9AAF7A9EED7E6944C3EBDBCB9E1BAC94954E6AF452DED592E01E622A5A97E1377D0D37A7C65EBC645DE6551A57BF4D699F88B096BB1C429A5B731DC1B6DC7E9E4F56FA7541B4932A6AA964BAD36C315A6A3AC7C9AD2360264C8C9061A7959399C143FF3AAA; __RequestVerificationToken=TysnDQx8eGRrBdaW6KOLlYJiBvHoiwRm5wAG_w9WowdPQdXBzLPwtAeZHxSoTFpMnCn1BonFbGJtkHGSVon7cO0s3tc1; ClientStreamAuthCookie=a519a494-56c2-41f3-9fa8-011df68f4acf",
        #'Cookie': "pageRows=10; ASP.NET_SessionId=nt5s1ebtlxuaobchtovnsxsr; PwCGTSiPOWER=8A8BFAEA5E4DEA97B75637DABDAB1CFA9FD3DF769210CCED4029D56D4D55B36D17377AB32BEF0CE94A9C1700E67AFFFA559A2A8DDB76ED05AAC64F1AA276E5DAC88ACC491299AD9A1E971C3C135D4819C456E2B971A373236571EABF312587C3FF2C782B7305B6F4BFEBB1E35A5A65F2DA4033BCAAA77A9DCE17E9BEA3D0DA63C0602B3871BCA621121266642FE76740DEF11B9CF6E330CCB64922AE7C844437AE7D8FF8E07FA3B7C1C3D182704E0CB51E79BC2D34C02767436A8955AD0F1AED9ADDFAD25069834D9BCAB25627FBF747EA7170800BB7A64AC0DE2EB3E22C7810990906FC44D8CDE612FA16A3FA5FAD1450F6EF84DAFD7F11E524891EAFC2148ABD5E1F418547536548CC580AF8775B5114A5853D585D94F11142D81CEC533274711A5DD029A5C161FB0E67D7CDD754D1C6C5FAD50C4D87672AC47413364C9A072217510A7506A6C5C67A1420F74F91EDCBBDF40516CF118708CA6A3E3D4A2382F8565A89; __RequestVerificationToken=TysnDQx8eGRrBdaW6KOLlYJiBvHoiwRm5wAG_w9WowdPQdXBzLPwtAeZHxSoTFpMnCn1BonFbGJtkHGSVon7cO0s3tc1; ClientStreamAuthCookie=a519a494-56c2-41f3-9fa8-011df68f4acf", 
        'Referer': "https://ipowerau.eps.ad.pwcinternal.com/gui/iexpense/frmEXExpenseFileReceiptsGet.aspx?intExpenseDtlId="+expenseIdDetail+"&intExpenseId="+expenseId+"&blnEditable=false"
        }
    response = requests.request("GET", url, verify=False, data=payload, headers=headers, params=querystring)
    
    #Check that expense report has attachment(s)
    if "intFileID" in response.text:
        #if only 1 attachment linked to expense id
        if isinstance(response.json()['NewDataSet']['Table'], dict) == True:
            fileId = response.json()['NewDataSet']['Table']['intFileID']
            chSelected = response.json()['NewDataSet']['Table']['chSelected']
            #check that attachment is linked to expenseIdDetail
            if chSelected == "True":
                attSingle += 1
                downloadURL = "https://ipowerau.eps.ad.pwcinternal.com/gui/iexpense/frmEXExpenseItemFileOpen.aspx?intFileId=" + fileId
                r = requests.get(downloadURL, headers=headers, stream=True, verify=False)
                open("/Volumes/GoogleDrive/My Drive/Enterprise digital/Clients - in TEAM DRIVE/Google/Zipline/Outputs_Final/"+ expenseId + "_" + expenseIdDetail + "_" + fileId + ".pdf", 'wb').write(r.content)
            else:
                attNone +=1
                
        #else there is multiple attachments
        else:
            #see how many link to the current expense Id detail
            attachmentCount = 0
            for i in range(0,len(response.json()['NewDataSet']['Table'])):
                chSelected = response.json()['NewDataSet']['Table'][i]['chSelected']
                if chSelected == "True":
                    fileId = response.json()['NewDataSet']['Table'][i]['intFileID']
                    check = chSelected
                    attachmentCount += 1    
            #if only 1 linked attachment, use the file Id to download
            if attachmentCount == 1:
                attSingle += 1
                downloadURL = "https://ipowerau.eps.ad.pwcinternal.com/gui/iexpense/frmEXExpenseItemFileOpen.aspx?intFileId=" + fileId
                r = requests.get(downloadURL, headers=headers, stream=True, verify=False)
                open("/Volumes/GoogleDrive/My Drive/Enterprise digital/Clients - in TEAM DRIVE/Google/Zipline/Outputs_Final/"+ expenseId + "_" + expenseIdDetail + "_" + fileId + ".pdf", 'wb').write(r.content)
            elif attachmentCount == 0:
                attNone += 1
            else:
                attMult += 1
    else:
        attNone +=1

print("All items parsed.")
print("----------------")
print("No attachments: "+ str(attNone))
print("Single attachment (saved): "+ str(attSingle))
print("Multiple attachments: "+ str(attMult))    
print("----------------")
print("Complete")

Opening File...
File Opened.
All items parsed.s parsed
----------------
No attachments: 19960
Single attachment (saved): 10319
Multiple attachments: 1321
----------------
Complete
